<a href="https://colab.research.google.com/github/marcosvnespolo/Previsao_Inadimplencia/blob/main/Previs%C3%A3o_Inadimpl%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sobre o Projeto

Uma empresa de cartão de crédito forneceu um dataset que inclui dados financeiros e demográficos de uma amostra de 30.000 titulares de contas. Os dados estão apresentados em nível conta de crédio, ou seja, cada linha do dataset representa uma conta de um cliente distinto.
O objetivo do projeto é desenvolver um modelo que preveja se uma conta ficará inadimplente no próximo mês, de acordo com dados demográficos e históricos.

Os dados já foram preparados e estão disponíveis na pasta do projeto.

In [1]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Projetos Portfólio/Projetos DS com Python/Projeto 1/default_of_credit_card_clients__courseware_version_1_21_19.xls')

# Inspecionando os Dados

In [2]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
df.shape

(30000, 25)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  object
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  int64 
 3   EDUCATION                   30000 non-null  int64 
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_1                       30000 non-null  object
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

In [5]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

Aparentemente a coluna ID apresenta o nome de cada cliente na base de dados, porém, podemos confirmar esta informação através do método .nunique()

In [6]:
df['ID'].nunique()


29687

O número de ID's únicos é inferior ao número de linhas, desta forma, o ID não é um identificador único, como sugerido anteriormente, sabemos então que há alguma duplicação de ID's.

Porém, quantos ID's estão duplicados? É possível começar a responder essa pergunta através do método .value_counts().

In [7]:
contagem_id = df['ID'].value_counts()

In [8]:
contagem_id.head(10)

ad23fe5c-7b09    2
1fb3e3e6-a68d    2
89f8f447-fca8    2
7c9b7473-cc2f    2
90330d02-82d9    2
2a793ecf-05c6    2
75938fec-e5ec    2
7be61027-a493    2
a3a5c0fc-fdd6    2
b44b81b2-7789    2
Name: ID, dtype: int64

Podemos utilizar novamente o método .value_counts() para identificar quantos valores são únicos ou duplicados.

In [9]:
contagem_id.value_counts()

1    29374
2      313
Name: ID, dtype: int64

# Examinando ID's duplicados

No passo anterior, foi observado que os ID's duplicados não aparecem mais do que duas vezes, utilizarei esta informação para encontrar e avaliá-los

In [10]:
# Visualizando os 15 primeiros ID's duplicados
id_duplicados = contagem_id == 2
id_duplicados[0:15]

ad23fe5c-7b09    True
1fb3e3e6-a68d    True
89f8f447-fca8    True
7c9b7473-cc2f    True
90330d02-82d9    True
2a793ecf-05c6    True
75938fec-e5ec    True
7be61027-a493    True
a3a5c0fc-fdd6    True
b44b81b2-7789    True
998fa9b2-b341    True
69566a6b-6156    True
4e2380e6-a8cf    True
b87bf8f3-d704    True
4f95b36b-ab10    True
Name: ID, dtype: bool

In [11]:
in_dupl_index = contagem_id.index[id_duplicados]
in_dupl_index

Index(['ad23fe5c-7b09', '1fb3e3e6-a68d', '89f8f447-fca8', '7c9b7473-cc2f',
       '90330d02-82d9', '2a793ecf-05c6', '75938fec-e5ec', '7be61027-a493',
       'a3a5c0fc-fdd6', 'b44b81b2-7789',
       ...
       '4f249cbc-5e9c', '40e75290-0f59', 'fc73f07e-eb96', '2a8ad33f-fa9c',
       'cdae2be5-8ec4', 'c69162db-4864', '73ea498f-44b2', '327d06d5-ce7f',
       'd13465a5-a9e0', '841ae407-dc4c'],
      dtype='object', length=313)

A series possui 313 entradas, exatamente o mesmo número obtido anteriormente.
No próximo passo, converterei a series para a forma de lista, pois este formato será útil em etapas posteriores do projeto.


In [12]:
lista_id_dupl = list(in_dupl_index)
len(lista_id_dupl)

313

In [13]:
# Verificando as primeiras entradas
lista_id_dupl[0:5]

['ad23fe5c-7b09',
 '1fb3e3e6-a68d',
 '89f8f447-fca8',
 '7c9b7473-cc2f',
 '90330d02-82d9']

In [14]:
df.loc[df['ID'].isin(lista_id_dupl[0:3]),:].head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
5033,89f8f447-fca8,320000,2,2,1,32,0,0,0,0,...,169371,172868,150827,8000,8000,5500,6100,6000,5000,0
5133,89f8f447-fca8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16727,1fb3e3e6-a68d,80000,1,2,2,33,2,2,0,0,...,27394,29922,31879,0,2000,2000,3000,2600,0,1
16827,1fb3e3e6-a68d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29685,ad23fe5c-7b09,50000,1,3,1,32,0,0,0,0,...,12882,8131,3983,3000,2871,1000,163,3983,3771,1
29785,ad23fe5c-7b09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O que parece estar acontecendo é que os ID's duplicados possuem uma linha válida, e outra composta apenas por 0, desta forma, nas próximas etapas do projeto, será necessário encontrar uma forma de xcluir estas linhas.

# Lidando com Valores Nulos

In [15]:
df_zero = df == 0

O objetivo, a partir de agora, é criar uma nova série booleana que identifique cada linha em que todos os elementos a partir da segunda coluna (Para não considerarmos a coluna ID) são iguais a zero.

In [16]:
linhas_zero = df_zero.iloc[:,1:].all(axis=1)

In [17]:
sum(linhas_zero)

315

O que o resultado acima nos diz é que 315 linhas tem zeros para cada coluna, exceto a primeira. Como este número é maior que o número de ID's duplicados (313), então podemos excluir estes registros sem maiores problemas.

Isto pode ser feito utilizando o código a seguir:

In [18]:
df_limpo = df.loc[~linhas_zero, :].copy()

O código acima retorna uma cópia dos dados que desejo manter, este é o motivo do uso do operador nógico not "~", pois desejo obter todas as observações que não tem zeros, e ":" foi utilziado para selecionar todas as colunas.

Utilizando o método .shape, verifico que o número de linhas é exatamente igual ao número de Id's únicos.

In [19]:
df_limpo.shape

(29685, 25)

In [20]:
df_limpo['ID'].nunique()

29685

Desta forma, conclui-se que os valores duplicados foram eliminados com sucesso, tendo em vista que o número de linhas de novo DataFram (df_limpo) é igual ao número de ID's únicos.